In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

In [3]:
import pytesseract
import re
import shutil
import os
import random
from IPython.display import display
try:
 from PIL import Image
except ImportError:
 import Image
import cv2
import numpy as np
from IPython.display import display, Image as IPImage

In [4]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)

#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

# Apply adaptive thresholding


In [5]:
img = cv2.imread('/content/uber_test4.PNG')

In [6]:
gray = get_grayscale(img)
# noise=remove_noise(gray) dont do it!
# thresh = thresholding(gray) nahh output is not good!
# opening = opening(gray)  dont use!
# canny = canny(gray)   dont use!

In [7]:
# inverted_binary_image = cv2.bitwise_not(gray)
# Image.fromarray(inverted_binary_image).show()

#inverting image color not helping

In [ ]:

img_pillow = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# image_path_in_colab="/content/ocr_test4.PNG"
extractedInformation = pytesseract.image_to_string(img_pillow)
print(extractedInformation)

# **Trying regular expression here**

In [ ]:
address_regex = re.compile(r'\b\d{1,2}:\d{2}\|\s*([^,]+),(\d+),\s*([^,]+),\s*([^,]+),\s*([^,]+),\s*([^,]+),\s*([^,]+ \d{6}),\s*([^,]+)\b')
# Extract addresses using the regular expression
matches = address_regex.findall(extractedInformation)
# Print the matched addresses
for match in matches:
    print(match)

In [ ]:
display(img_pillow)

Uber dy 2023

Here's your receipt for your ride, Punit

‘We hope you enjoyed your ride this moming.



Total 1969.00
‘Tipcharge z198954
Subtotal 1989.54
BIAL Airport South Toll 11550
Rounding 40.28
Promotion 213576
Payments

Cash



06/07/2023 12:42
Visitthe trio nage for more information, including invoices (where available)

‘The total of €1969.00 has a GST of 37.02 included.

‘Yourode with AKASH
License Plat: KAO2AJO4B6

Uber Go Sedan 41.37 kilometres | 1h 18
rmin(s)
1 11:23| The Forum Mall,21, Hosur Rd, Chikku Lakshmaiah Layout, Koramangala, Bengaluru, Kamataka 560095, India

12:41 | Kempegowda International Airport Bengaluru, 2 Terminal, Bengaluru, Kamataka 560300, India

Fares are inclusive of GST. Please download the tax invoice from the trip detail page for a full tax breakdown.

#**Trying NER here**

# **Using Spacy**

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
doc=nlp(extractedInformation)
for ent in doc.ents:
  if ent.label_== "DATE":
    temp=ent.text
  print(ent.text, "|", ent.label_, "|", spacy.explain(ent.label_))

# **Using Bert Base NER**

In [ ]:
!pip install transformers

In [10]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [ ]:

ner_results = nlp(extractedInformation)
print(ner_results)

# **Using Bert-large-NER**

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
ner_results = nlp(extractedInformation)
print(ner_results)

In [ ]:
def extract_address(ner_result):
    addresses = []
    current_address = []

    for token in ner_result:
        if token['entity'].startswith('B-LOC'):
            if current_address:
                addresses.append(" ".join(current_address))
                current_address = [token['word']]
            else:
                current_address.append(token['word'])
        elif token['entity'].startswith('I-LOC'):
            current_address.append(token['word'])

    if current_address:
        addresses.append(" ".join(current_address))

    return addresses

# Extract addresses
addresses = extract_address(ner_results)

# Print the addresses
for address in addresses:
    print(address)

In [14]:
def correctTokens(address):
  split_address=address.split()
  tokens = split_address
  # Combine '##' tokens with the preceding non-'##' element
  combined_tokens = []
  current_token = None

  for token in tokens:
      if token.startswith('##'):
          if current_token is not None:
              current_token += token[2:]  # Concatenate with the non-'##' part
      else:
          if current_token is not None:
              combined_tokens.append(current_token)
          current_token = token

  # Add the last token if it exists
  if current_token is not None:
      combined_tokens.append(current_token)

  # Replace the first non-'##' token with the combined tokens
  if combined_tokens:
      tokens[0] = combined_tokens[0]

  # Remove '##' tokens from the list
  tokens = [token for token in combined_tokens if not token.startswith('##')]

  # Print the result
  # print(tokens)
  result_string = ' '.join(tokens)

  # Print the result
  # print(result_string)
  return result_string

In [ ]:
correctTokens(addresses[9])

In [ ]:
corrected_addresses = []

# Iterate through the original addresses and store the corrected ones in the new list
for address in addresses:
    corrected_addresses.append(correctTokens(address))

# Print the corrected addresses
for address in corrected_addresses:
    print(address)

In [ ]:

indices_of_india = [i for i, x in enumerate(corrected_addresses) if "India" in x]

# Check if at least two occurrences of "India" are found
if len(indices_of_india) >= 2:
    # Extract the substring from the beginning to the first occurrence of "India"
    substring_before_first_india = corrected_addresses[:indices_of_india[0] + 1]

    # Extract the substring from the first occurrence of "India" to the second occurrence of "India"
    substring_between_indias = corrected_addresses[indices_of_india[0] + 1:indices_of_india[1] + 1]

first_address = '\n'.join(substring_before_first_india)
second_address= '\n'.join(substring_between_indias)

# Print the combined string
print(first_address)
print(second_address)
